<a href="https://colab.research.google.com/github/gvanathip/webscraping/blob/main/CarParkBKK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install requests

In [ ]:
pip install beautifulsoup4

In [ ]:
from bs4 import BeautifulSoup
import requests

In [ ]:
url1 = 'https://parkandridefinder.com/parking-2/'
url2 = 'https://parkandridefinder.com/parking-2/page/2/'
url3 = 'https://parkandridefinder.com/parking-2/page/3/'

In [ ]:
urls = [url1,url2,url3]

In [ ]:
a = []
for url in urls:
  res = requests.get(url)
  res.encoding = "utf-8"
  soup = BeautifulSoup(res.text, 'html.parser')
  print(len(soup.find_all('a')))
  for i in soup.find_all('a'):
    a.append(i)

530
477
188


In [ ]:
len(a)

1195

In [ ]:
links = []
for i in a:
  if i.get('href'):
    links.append(i.get('href'))

In [ ]:
correct_links = []
for link in links:
  if 'properties' in link:
    correct_links.append(link)

In [ ]:
correct_links = list(set(correct_links))

In [ ]:
len(correct_links)

186

In [ ]:
content = []
for link in correct_links:
  test = requests.get(link)
  test.encoding = "utf-8"
  soup_test = BeautifulSoup(test.text, 'html.parser')

  for i in soup_test.find_all('a'):
    if i.get('href'):
      if 'maps.google' in i.get('href'):
        lat = i.get('href').split('=')[-1].split(',')[0]
        lng = i.get('href').split('=')[-1].split(',')[1]
  
  name = soup_test.title.string.split(' – ')[0]

  body = []
  for i in soup_test.find_all('span'):
    if i.string:
      body.append(i.string)

  body = body[body.index('ประเภท : '):]

  flag = dict()
  for i in range(len(body)):
    if body[i][-3:] == ' : ':
      flag[i] = 'title'
    else:
      flag[i] = 'detail'
    
  id = dict()
  for i in range(len(body)):
    id[i] = body[i]
  
  info = dict()
  for i in range(len(id)):
    if i+3 < len(id):
      if flag[i] == 'title' and flag[i+1] == 'detail':
        info[id[i]] = id[i+1]
      elif i+3 < len(id):
        if flag[i] == 'title' and flag[i+1] == 'detail' and flag[i+2] == 'detail':
          info[id[i]] = id[i+1]+id[i+2]

  
  info['name'] = name
  info['lat'] = lat
  info['lng'] = lng

  content.append(info)

In [ ]:
df = pd.DataFrame(content)

In [261]:
df.head()

,ประเภท :,รายชั่วโมง :,รายวัน :,รายเดือน :,ระยะเดินเท้าไปสถานี :,จำนวนที่จอดรถ :,สถานที่ตั้ง :,เวลาทำการ :,รายละเอียดติดต่อ :,name,lat,lng,จำนวนที่จอดมอเตอร์ไซต์ :,ย่าน :,สถานี :
0,ลานจอดรถ,-,30 บาท,"1,000 บาท",300 เมตร,250,อยู่ในซอย,5.30-21.00,ติดต่อ ณ ที่จอดรถ,วัดประดู่บางจาก,13.722737000000008,100.4557810158692,NaN,NaN,NaN
1,ลานจอดรถ,NaN,NaN,"2,000 บาท",ุ400 เมตร,10,อยู่ในซอย,24 ชั่วโมง,สนใจติดต่อ 0-97045-1720,Wutthakat parking,13.715344,100.468489,NaN,NaN,NaN
2,ลานจอดรถ,ฟรี,ฟรี,ฟรี,50 เมตร,"1,250",ติดถนนใหญ่,05.00-01.00,NaN,ลานจอดรถ BTS หมอชิต,13.80373,100.554885,-,NaN,NaN
3,อาคารจอดรถ,NaN,NaN,"2,200",450 เมตร,NaN,อยู่ในซอย,06:00 - 24:00 Hrs.,สนใจติดต่อ 089-939-7037 คุณกาแฟ หรือ 081-373-3...,Bangkok Hotel Lotus Sukhumvit,13.7334628,100.5670827,NaN,NaN,NaN
4,อาคารจอดรถ,2 ชั่วโมงแรก 20 บาท ชั่วโมงถัดไป ชั่วโมงละ 20 บาท,- รายวัน 1-4 ชั่วโมงแรก 30 บาท/ชั่วโมง - ชั่วโ...,"4,000 บาท","1,000 เมตร",250 คัน,อยู่ในซอย,24 ชั่วโมง,สอบถาม โทร. 095-710-1731,loco 24 parking DRAGONTOWN,13.741314476426687,100.52399917417029,NaN,NaN,NaN


In [262]:
pip install xlsxwriter

     |████████████████████████████████| 153kB 4.1MB/s 


In [264]:
out_path = "/content/drive/MyDrive/public_database/CarParkBKK.xlsx"

writer = pd.ExcelWriter(out_path, engine='xlsxwriter')
df.to_excel(writer, index=False)
writer.save() 